#### Root Validators

Validation can also be performed on the entire model's data.

In [1]:
from pydantic import BaseModel, ValidationError, root_validator

In [2]:
class UserModel(BaseModel):
    username: str
    password1: str
    password2: str
    
    @root_validator(pre=True)
    def check_card_number_omitted(cls, values):
        assert "card_number" not in values, "card_number should not be included"
        return values
    
    @root_validator
    def check_passwords_match(cls, values):
        pw1, pw2 = values.get("password1"), values.get("password2")
        if pw1 is not None and pw2 is not None and pw1 != pw2:
            raise ValueError("passwords do not match")
        return values

In [3]:
print(f"{UserModel(username='scolvin', password1='zxcvbn', password2='zxcvbn') = }")

UserModel(username='scolvin', password1='zxcvbn', password2='zxcvbn') = UserModel(username='scolvin', password1='zxcvbn', password2='zxcvbn')


In [4]:
try:
    print(f"{UserModel(username='scolvin', password1='zxcvbn', password2='zxcvbn2') = }")
except ValidationError as e:
    print(e)

1 validation error for UserModel
__root__
  passwords do not match (type=value_error)


In [5]:
try:
    m = UserModel(
        username="scolvin",
        password1="zxcvbn",
        password2="zxcvbn",
        card_number="1234",
    )
    print(m)
except ValidationError as e:
    print(e)

1 validation error for UserModel
__root__
  card_number should not be included (type=assertion_error)


As with field validators, root validators can have `pre=True`, in which case they're called before field validation occurs (and are provided with the raw input data), or `pre=False` (the default), in which case they're called after field validation.

Field validation will not occur if `pre=True` root validators raise an error. As with field validators, "post" (i.e. `pre=False`) root validators by default will be called even if prior validators fail; this behaviour can be changed by setting the `skip_on_failure=True` keyword argument to the validator. The `values` argument will be a dict containing the values which passed field validation and field defaults where applicable.